In [1]:
import pandas as pd
import numpy as np
import warnings
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import callbacks
from keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from keras.layers import Dense, Dropout
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

warnings.filterwarnings(action='ignore')

In [2]:
df = pd.read_csv("modified_train_0412.csv")
x_train = df.drop(columns=['loan_status', 'addr_state'])
y_train = df['loan_status']

In [3]:
nan_indices = np.isnan(x_train).any(axis=1)
x_train = x_train[~nan_indices]
y_train = y_train[~nan_indices]

In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x_train)

# PCA로 차원 축소
pca = PCA(n_components=90)
X_pca = pca.fit_transform(X_scaled)
x_train = X_pca.astype('float32')
# stratify로 학습에 용이하게 비율 유지
X_Train, X_Validation, Y_Train, Y_Validation = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)

In [6]:
Y_Train=to_categorical(Y_Train, 2).astype(int)
Y_Validation=to_categorical(Y_Validation, 2).astype(int)

In [10]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [9]:
# layers
model = keras.Sequential([
    Dense(45, input_dim=90,activation='relu'),
    Dropout(0,2),
    Dense(45,activation='relu'),
    Dropout(0.2),
    Dense(45, activation='relu'),
    Dense(2,activation='softmax'),
])

# optimizer
optimizer = Adam(learning_rate=0.01)

# compile
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy", f1_score, recall, precision])

# train
epochs = 100
batch_size = 200000
early_stopping = EarlyStopping(min_delta=0.01)
model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=epochs)
# validation
valid_loss, valid_acc, valid_f1, valid_recall, valid_precision = model.evaluate(X_Validation, Y_Validation, callbacks=[early_stopping])
print("===================================")
print("Validation accuracy:", valid_acc)
print("Validation F1-score:", valid_f1)
print("Validation recall:", valid_recall)
print("Validation precesion:", valid_precision)

Epoch 1/100


2024-04-15 12:00:55.137493: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


5/5 [==============================] - 2s 212ms/step - loss: 0.5641 - accuracy: 0.7564 - f1_score: 0.7614 - recall: 0.7614 - precision: 0.7614
Epoch 2/100
5/5 [==============================] - 1s 158ms/step - loss: 0.4882 - accuracy: 0.7961 - f1_score: 0.7958 - recall: 0.7958 - precision: 0.7958
Epoch 3/100
5/5 [==============================] - 1s 154ms/step - loss: 0.4745 - accuracy: 0.7961 - f1_score: 0.7962 - recall: 0.7962 - precision: 0.7962
Epoch 4/100
5/5 [==============================] - 1s 155ms/step - loss: 0.4688 - accuracy: 0.7961 - f1_score: 0.7959 - recall: 0.7959 - precision: 0.7959
Epoch 5/100
5/5 [==============================] - 1s 157ms/step - loss: 0.4661 - accuracy: 0.7961 - f1_score: 0.7961 - recall: 0.7961 - precision: 0.7961
Epoch 6/100
5/5 [==============================] - 1s 156ms/step - loss: 0.4637 - accuracy: 0.7961 - f1_score: 0.7962 - recall: 0.7962 - precision: 0.7962
Epoch 7/100
5/5 [==============================] - 1s 170ms/step - loss: 0.4621 - 